In [1]:
!python -V

Python 3.9.15


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error


In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

2023/01/15 17:10:05 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/4', creation_time=1673782805020, experiment_id='4', last_update_time=1673782805020, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [8]:


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



In [9]:
df_train = read_dataframe('/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']


In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7587151974993125

### Using XgBoost

In [16]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=30,   #1000
            evals=[(valid, 'validation')],
            early_stopping_rounds=10   #50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}



In [19]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[17:16:40] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                     
[0]	validation-rmse:19.37377                                                                                                                               
[1]	validation-rmse:17.75909                                                                                                                               
[2]	validation-rmse:16.32863                                                                                                                               
[3]	validation-rmse:15.06383                                                                                                                               
[4]	validation-rmse:13.95011                                                                                                                               
[5]	validation-rmse:12.97114                                    

[20]	validation-rmse:8.04924                                                                                                                               
[21]	validation-rmse:7.90409                                                                                                                               
[22]	validation-rmse:7.77602                                                                                                                               
[23]	validation-rmse:7.66247                                                                                                                               
[24]	validation-rmse:7.56083                                                                                                                               
[25]	validation-rmse:7.47134                                                                                                                               
[26]	validation-rmse:7.39171                                    

[10]	validation-rmse:6.76781                                                                                                                               
[11]	validation-rmse:6.74024                                                                                                                               
[12]	validation-rmse:6.72513                                                                                                                               
[13]	validation-rmse:6.71035                                                                                                                               
[14]	validation-rmse:6.70331                                                                                                                               
[15]	validation-rmse:6.69603                                                                                                                               
[16]	validation-rmse:6.69199                                    

[0]	validation-rmse:9.44301                                                                                                                                
[1]	validation-rmse:7.22160                                                                                                                                
[2]	validation-rmse:6.83856                                                                                                                                
[3]	validation-rmse:6.75336                                                                                                                                
[4]	validation-rmse:6.72088                                                                                                                                
[5]	validation-rmse:6.71056                                                                                                                                
[6]	validation-rmse:6.70634                                     

[21]	validation-rmse:7.75153                                                                                                                               
[22]	validation-rmse:7.61853                                                                                                                               
[23]	validation-rmse:7.50072                                                                                                                               
[24]	validation-rmse:7.39632                                                                                                                               
[25]	validation-rmse:7.30360                                                                                                                               
[26]	validation-rmse:7.22160                                                                                                                               
[27]	validation-rmse:7.14798                                    

...

[22]	validation-rmse:6.70613                                                                                                                               
[23]	validation-rmse:6.69370                                                                                                                               
[24]	validation-rmse:6.68297                                                                                                                               
[25]	validation-rmse:6.67571                                                                                                                               
[26]	validation-rmse:6.67060                                                                                                                               
[27]	validation-rmse:6.66546                                                                                                                               
[28]	validation-rmse:6.65892                                    

[12]	validation-rmse:6.59357                                                                                                                               
[13]	validation-rmse:6.58720                                                                                                                               
[14]	validation-rmse:6.58377                                                                                                                               
[15]	validation-rmse:6.58140                                                                                                                               
[16]	validation-rmse:6.57947                                                                                                                               
[17]	validation-rmse:6.57719                                                                                                                               
[18]	validation-rmse:6.57548                                    

[2]	validation-rmse:16.11156                                                                                                                               
[3]	validation-rmse:14.80753                                                                                                                               
[4]	validation-rmse:13.66397                                                                                                                               
[5]	validation-rmse:12.66632                                                                                                                               
[6]	validation-rmse:11.79812                                                                                                                               
[7]	validation-rmse:11.04630                                                                                                                               
[8]	validation-rmse:10.39282                                    

[23]	validation-rmse:9.31629                                                                                                                               
[24]	validation-rmse:9.12627                                                                                                                               
[25]	validation-rmse:8.94979                                                                                                                               
[26]	validation-rmse:8.78632                                                                                                                               
[27]	validation-rmse:8.63464                                                                                                                               
[28]	validation-rmse:8.49419                                                                                                                               
[29]	validation-rmse:8.36424                                    

KeyboardInterrupt: 

## experimenting using autolog

In [20]:
mlflow.xgboost.autolog(disable=True)


In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,  #1000
        evals=[(valid, 'validation'],
        early_stopping_rounds=10 #50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


[17:21:40] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95635
[2]	validation-rmse:16.59103
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29029
[5]	validation-rmse:13.32807
[6]	validation-rmse:12.47571
[7]	validation-rmse:11.72138
[8]	validation-rmse:11.05825
[9]	validation-rmse:10.47535
[10]	validation-rmse:9.96449
[11]	validation-rmse:9.51756
[12]	validation-rmse:9.12641
[13]	validation-rmse:8.78569
[14]	validation-rmse:8.49089
[15]	validation-rmse:8.23445
[16]	validation-rmse:8.01207
[17]	validation-rmse:7.81886
[18]	validation-rmse:7.65169
[19]	validation-rmse:7.50672
[20]	validation-rmse:7.38153
[21]	validation-rmse:7.27299
[22]	validation-rmse:7.17863
[23]	validation-rmse:7.09718
[24]	validation-rmse:7.02603
[25]	validation-rmse:6.96344
[26]	validation-rmse:6.90935
[27]	validation-rmse:6.86285
[28]	validation-rmse:6.82157
[29]	validation-rmse:6.78486
[30]	valida

[273]	validation-rmse:6.38760
[274]	validation-rmse:6.38752
[275]	validation-rmse:6.38711
[276]	validation-rmse:6.38693
[277]	validation-rmse:6.38679
[278]	validation-rmse:6.38631
[279]	validation-rmse:6.38589
[280]	validation-rmse:6.38573
[281]	validation-rmse:6.38517
[282]	validation-rmse:6.38490
[283]	validation-rmse:6.38479
[284]	validation-rmse:6.38458
[285]	validation-rmse:6.38413
[286]	validation-rmse:6.38405
[287]	validation-rmse:6.38383
[288]	validation-rmse:6.38357
[289]	validation-rmse:6.38302
[290]	validation-rmse:6.38281
[291]	validation-rmse:6.38251
[292]	validation-rmse:6.38235
[293]	validation-rmse:6.38202
[294]	validation-rmse:6.38163
[295]	validation-rmse:6.38126
[296]	validation-rmse:6.38063
[297]	validation-rmse:6.38038
[298]	validation-rmse:6.37990
[299]	validation-rmse:6.37973
[300]	validation-rmse:6.37959
[301]	validation-rmse:6.37926
[302]	validation-rmse:6.37923
[303]	validation-rmse:6.37911
[304]	validation-rmse:6.37905
[305]	validation-rmse:6.37882
[306]	vali

[547]	validation-rmse:6.33673
[548]	validation-rmse:6.33664
[549]	validation-rmse:6.33674
[550]	validation-rmse:6.33657
[551]	validation-rmse:6.33638
[552]	validation-rmse:6.33626
[553]	validation-rmse:6.33601
[554]	validation-rmse:6.33603
[555]	validation-rmse:6.33590
[556]	validation-rmse:6.33570
[557]	validation-rmse:6.33575
[558]	validation-rmse:6.33562
[559]	validation-rmse:6.33544
[560]	validation-rmse:6.33541
[561]	validation-rmse:6.33536
[562]	validation-rmse:6.33521
[563]	validation-rmse:6.33495
[564]	validation-rmse:6.33483
[565]	validation-rmse:6.33465
[566]	validation-rmse:6.33451
[567]	validation-rmse:6.33442
[568]	validation-rmse:6.33436
[569]	validation-rmse:6.33431
[570]	validation-rmse:6.33419
[571]	validation-rmse:6.33422
[572]	validation-rmse:6.33411
[573]	validation-rmse:6.33407
[574]	validation-rmse:6.33394
[575]	validation-rmse:6.33374
[576]	validation-rmse:6.33359
[577]	validation-rmse:6.33329
[578]	validation-rmse:6.33320
[579]	validation-rmse:6.33304
[580]	vali

[821]	validation-rmse:6.31206
[822]	validation-rmse:6.31200
[823]	validation-rmse:6.31205
[824]	validation-rmse:6.31194
[825]	validation-rmse:6.31188
[826]	validation-rmse:6.31180
[827]	validation-rmse:6.31174
[828]	validation-rmse:6.31150
[829]	validation-rmse:6.31147
[830]	validation-rmse:6.31113
[831]	validation-rmse:6.31115
[832]	validation-rmse:6.31116
[833]	validation-rmse:6.31122
[834]	validation-rmse:6.31111
[835]	validation-rmse:6.31107
[836]	validation-rmse:6.31105
[837]	validation-rmse:6.31100
[838]	validation-rmse:6.31083
[839]	validation-rmse:6.31074
[840]	validation-rmse:6.31056
[841]	validation-rmse:6.31045
[842]	validation-rmse:6.31045
[843]	validation-rmse:6.31039
[844]	validation-rmse:6.31026
[845]	validation-rmse:6.31024
[846]	validation-rmse:6.31018
[847]	validation-rmse:6.31003
[848]	validation-rmse:6.30992
[849]	validation-rmse:6.30993
[850]	validation-rmse:6.30999
[851]	validation-rmse:6.30970
[852]	validation-rmse:6.30967
[853]	validation-rmse:6.30965
[854]	vali

FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'

In [22]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-01.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'